In [302]:
import numpy as np, time
import os,sys, csv, pickle

In [303]:
import cv2
from collections import Counter, OrderedDict, defaultdict

In [304]:
#path = r'D:\code\rough'
#os.chdir(path)
%pwd

'D:\\imagepro\\deep_version\\extraction_code\\1_Image9'

In [305]:
os.chdir("D:/imagepro/deep_version/extraction_code/1_Image9")

In [306]:
print(os.getcwd())

D:\imagepro\deep_version\extraction_code\1_Image9


In [307]:
#orig_img = cv2.imread('w9-form.jpg',0)
orig_img = cv2.imread('1_Image9.jpg',0)

ret, thresh = cv2.threshold(orig_img, 190, 255 , 0)
img = thresh.copy()
img2 = img.copy()
cv2.imwrite('thresh.jpg', img)

True

In [308]:
#img[200,-5:-1], img[200,]

In [309]:
print(np.count_nonzero(img) / float(img.shape[0] *img.shape[1]) )

0.93796492052306


In [310]:
def font_is_dark(img):
    h, w = img.shape
    
    f_count = Counter(img[0])
    l_count = Counter(img[h-1])
    # if number of white pixels >= 50% then it's a white background with dark font
    if np.count_nonzero(img) / float(h * w) >= 0.5:
        dark = False
    else:
        dark = True

    return dark
    

In [311]:
if font_is_dark(thresh):
    val = 255
else:
    val = 0
val

0

In [312]:
arr1 = np.zeros((15,1))
arr1[0:10,0] = 10
#arr1[10:15,0] = 0
np.count_nonzero(arr1[:15,0] == 0)


5

In [313]:
jump, v_span, h_span, v_clean, h_clean, RECT_PERC, ARC_PERC, AREA_PERC = 0, 30, 30, 0.55, 0.55, 0.9, 0.7, 0.35

In [314]:
# logic for top left
DIAG = False
coord = []

h, w = img.shape
print(h, w)
for i in range(h):
    if DIAG:
        if i < 290 or i > 296:
            continue
        print('i', i)
        temp_horiz = False

    if h - i < v_span:
        break
    end_vert = i + v_span
    vert_stem = False
    for j in range(w): 
        if w - j < h_span:
            break
        horiz_stem = False
        end_horiz = j + h_span
        #print('end ', end_vert)
        if all(img[i:end_vert,j] == val):
            vert_stem = True    
            if DIAG:
                print('values: ', j , img[i:end_vert,j])
        else:
            if jump > 0:
                ext = end_vert + jump
                if end_vert> h:
                    break
                if np.count_nonzero(img[i:ext, j] == val) >=  v_clean * v_span:
                    #print('here',i,ext,j, v_span, np.count_nonzero(img[i:ext, j] != val))
                    vert_stem = True

        if vert_stem and all(img[i,j:end_horiz] == val):
            horiz_stem = True
        else:
            if jump > 0:
                ext = end_horiz + jump
                if end_horiz > w:
                    break            
                if np.count_nonzero(img[i, j:ext] == val) >= h_clean * h_span:
                    #print('here 2',i,ext,j, h_span, np.count_nonzero(img[i, j:ext] == val))
                    horiz_stem = True
                
        if vert_stem and horiz_stem:
            coord.append([j,i])
            if DIAG:
                temp_horiz = horiz_stem
        vert_stem, horiz_stem = False, False
    if DIAG:
        print(vert_stem, temp_horiz)
            
        

1548 1188


In [315]:
# logic for bottom right
DIAG = False
coord_l = []

h, w = img.shape
print(h, w)
for i in range(h-1,-1, -1):
    if DIAG:
        if i < 290 or i > 296:
            continue
        print('i', i)
        temp_horiz = False

    if i + 1 < v_span:
        continue
    end_vert = i - v_span + 1
    vert_stem = False
    for j in range(w-1, -1, -1): 
        if j + 1 < h_span:
            continue
        horiz_stem = False
        end_horiz = j - h_span + 1
        #print('end ', end_vert)
        if all(img[end_vert:i+1,j] == val):
            vert_stem = True    
            if DIAG:
                print('values: ', j , img[end_vert:i+1,j])
        else:
            if jump > 0:
                ext = end_vert - jump
                if end_vert < 0:
                    continue
                if np.count_nonzero(img[ext:i+1, j] == val) >=  v_clean * v_span:
                    #print('here',i,ext,j, v_span, np.count_nonzero(img[i:ext, j] != val))
                    vert_stem = True

        if vert_stem and all(img[i,end_horiz:j+1] == val):
            horiz_stem = True
        else:
            if jump > 0:
                ext = end_horiz - jump
                if end_horiz < 0:
                    continue            
                if np.count_nonzero(img[i, ext:j+1] == val) >= h_clean * h_span:
                    #print('here 2',i,ext,j, h_span, np.count_nonzero(img[i, j:ext] == val))
                    horiz_stem = True
                
        if vert_stem and horiz_stem:
            coord_l.append([j,i])
            if DIAG:
                temp_horiz = horiz_stem
        vert_stem, horiz_stem = False, False
    if DIAG:
        print(vert_stem, temp_horiz)
            
        

1548 1188


In [316]:
len(coord), len(coord_l)

(68, 50)

In [317]:
'''
coord_dict = defaultdict(list)
for k, v in zip(y_coord, x_coord):
    coord_dict[k].append(v)

#coord_dict
len(coord_dict)
'''

'\ncoord_dict = defaultdict(list)\nfor k, v in zip(y_coord, x_coord):\n    coord_dict[k].append(v)\n\n#coord_dict\nlen(coord_dict)\n'

In [318]:
'''
[k for k in coord_dict.keys() if k <= 126]
x_coord[(x_coord) <= 189]
x, y =coord[0]
print(x, y)
[82, 119] in coord    
'''

'\n[k for k in coord_dict.keys() if k <= 126]\nx_coord[(x_coord) <= 189]\nx, y =coord[0]\nprint(x, y)\n[82, 119] in coord    \n'

In [319]:
x = np.array(([3,4],[4,2],[3,3]))
#y = np.transpose(x)
#np.reshape(x, -1), np.reshape(y, -1)
z = np.ravel(x,order='F')
#z = np.flip(np.flip(x, 0), 1)
z = np.flip(z, 0)
w = np.argmax(z)
print(w, z)

ind = np.unravel_index(w, x.shape, 'F')
print(ind)
#np.argmax(x, axis=None)
ymax, ymin = 33, 31
xmax, xmin = 21, 20
y = abs(ymax - ind[0])
x = abs(xmax - ind[1])
y, x


2 [3 2 4 3 4 3]
(2, 0)


(31, 21)

In [320]:
# this is the 2nd of two approaches to trim redundant coordinates. this is very quick.
def trim_t_coords(coord, polarity):
    DIAG = False
    final = []
    
    h, w = img.shape
    master = np.zeros([h, w])
    lookup = np.zeros([h, w])
    for vals in coord:
        x, y = vals
        master[y, x] = 1
    
    for ix, vals in enumerate(coord):
        list_pop = False
        if ix % 1000 == 0:
            print(ix+1, 'out of %d vals %s' %(len(coord),vals) )    
        if DIAG:
            print('vals: ', vals)

        x , y = vals

        if lookup[y, x] <= -1:
            if DIAG:
                print('hmm')
                print(lookup[332, 75], y , x)
            continue
        
        max_x = x + (h_span - 1) * polarity
        max_y = y + (v_span - 1) * polarity
        if max_x > w:
            max_x = w * max(0, polarity)
        if max_y > h:
            max_y = h * max(0, polarity)
        cut_mat = master[y:max_y+1, x:max_x+1]
        if np.count_nonzero(cut_mat) == 1:
            t_x, t_y = x ,y
            #final.append((x, y))
            if DIAG:
                print('1')
        elif [max_x, max_y] in coord:
            t_x ,t_y = max_x, max_y
            list_pop = True
            #final.append((max_x, max_y))
            if DIAG:
                print('2')            
        else:  
            if DIAG:
                print('point 3')  

            list_pop = True
            ravel = np.ravel(cut_mat, order='F')
            flip = np.flip(ravel, 0)
            maxind = np.argmax(flip)
            ind = np.unravel_index(maxind , cut_mat.shape, 'F')
            y_c = max_y - ind[0] * max(-1, polarity)
            x_c = max_x - ind[1] * max(-1, polarity)
            t_x, t_y = x_c, y_c
            #final.append((x_c, y_c))
        
        if list_pop:
            lgth = final.count((x, y))
            [final.remove((x,y)) for i in range(lgth)]
            if DIAG:
                print('oh!')
 
        final.append((t_x, t_y))
        lookup[t_y, t_x] = 5000
        lookup[y:max_y+1, x:max_x+1] += -1
        
        if DIAG:
            print(t_x, t_y)
            print(lookup[332, 75])
            
    return final

In [321]:
# this is the 2nd of two approaches to trim redundant coordinates. this is very quick.
def trim_b_coords(coord, polarity):
    DIAG = False
    final = []
    
    h, w = img.shape
    master = np.zeros([h, w])
    lookup = np.zeros([h, w])
    for vals in coord:
        x, y = vals
        master[y, x] = 1
    
    for ix, vals in enumerate(coord):
        list_pop = False
        if ix % 1000 == 0:
            print(ix+1, 'out of %d vals %s' %(len(coord),vals) )    
        if DIAG:
            print('vals: ', vals)

        x , y = vals

        if lookup[y, x] <= -1:
            if DIAG:
                print('hmm')
                print(lookup[332, 75], y , x)
            continue
        
        max_x = x + (h_span - 1) * polarity
        max_y = y + (v_span - 1) * polarity
        
        if max_x > w or max_x < 0:
            max_x = w * max(0, polarity)
        if max_y > h or max_y < 0:
            max_y = h * max(0, polarity)
        cut_mat = master[max_y:y+1, max_x:x+1]
        if np.count_nonzero(cut_mat) == 1:
            t_x, t_y = x ,y
            #final.append((x, y))
            if DIAG:
                print('1')
        elif [max_x, max_y] in coord:
            t_x ,t_y = max_x, max_y
            list_pop = True
            #final.append((max_x, max_y))
            if DIAG:
                print('2')            
        else:  
            if DIAG:
                print('point 3')  

            list_pop = True
            ravel = np.ravel(cut_mat, order='F')
            #flip = np.flip(ravel, 0)
            maxind = np.argmax(ravel)
            ind = np.unravel_index(maxind , cut_mat.shape, 'F')
            y_c = max_y - ind[0] * max(-1, polarity)
            x_c = max_x - ind[1] * max(-1, polarity)
            t_x, t_y = x_c, y_c
            #final.append((x_c, y_c))
        
        if list_pop:
            lgth = final.count((x, y))
            [final.remove((x,y)) for i in range(lgth)]
            if DIAG:
                print('oh!')
 
        final.append((t_x, t_y))
        lookup[t_y, t_x] = 5000
        lookup[max_y:y+1, max_x:x+1] += -1
        
        if DIAG:
            print(t_x, t_y)
            print(lookup[332, 75])
            
    return final

In [322]:
#len(trim_coords(coord))
w,h 

(2, 1548)

In [323]:
st = time.time()
#print(len(coord), val)
#coord
final = set(trim_t_coords(coord, +1))
final_l = set(trim_b_coords(coord_l, -1))
print('time: ' , time.time() - st, '\n')

#st = time.time()
#final1 = set(trim1_coords(coord))
#print(len(final))
#print('time: ' , time.time() - st)

1 out of 68 vals [412, 124]
1 out of 50 vals [1118, 440]
time:  0.005000591278076172 



In [324]:
set1 = sorted(final.copy(), key=lambda x: (x[0], x[1]))
set2 = sorted(final_l.copy(), key=lambda x: (x[0], x[1]))
lst2 = set2.copy()

arr1 = np.repeat(0,100000)
arr2 = np.array([0,0])
arr3 = np.array([1,0])
arr4 = np.array([0,1])
arrf = np.hstack([arr1, arr2, arr3, arr4])

#list((arrf == 1)).count(True)
np.count_nonzero(arrf == 0)

d1 = defaultdict(int)
d2 = OrderedDict()
d3 = {int}
for x, y in set2:
    d1[x] +=  y
len(d1)

2

In [325]:
def is_block(img, area, val):
    if np.count_nonzero(img == val) >= area * RECT_PERC:
        #print('cnt ', np.count_nonzero(img == val) , area )
        return True
    else:
        #print('cnt ', np.count_nonzero(img == val) , area )
        return False

def dense_area(surface, val):
    temp = surface.ravel()
    if np.count_nonzero(temp == val) > len(temp) * AREA_PERC:
        return True
    else:
        return False

def count_dark_pixels(surface, val):
    diag = False
    max_y , max_x = surface.shape
    y_len = int(max_y * 0.33)
    x_len = int(max_x * 0.33)
    hit, total = 0, 0
    arc_list = ['top-left', 'top-right', 'bot-left', 'bot-right']
    arc_y = [ (0, y_len) , (0, y_len) , (max_y-y_len, max_y), (max_y-y_len, max_y)] 
    arc_x = [(0, x_len), (max_x - x_len, max_x) , (0, x_len), (max_x - x_len, max_x)]
    #print('surface ', surface)
    for ix, arc in enumerate(arc_list):
        y_axis = arc_y[ix]
        x_axis = arc_x[ix]
        
        a, b = shape_count(surface[y_axis[0]:y_axis[1], x_axis[0]:x_axis[1]], val, arc, y_len, x_len)
        if diag:
            print('a, b' , a ,b)
        hit += a
        total += b
    
    return (hit, total)
    
def shape_count(surface, val, shape, i_max, j_max):
    #print('surface: ', surface)
    diag = False
    if shape == 'top-left':
        y_st, y_end, x_st, x_end  = 0, i_max, 0, j_max
        y_sign, x_sign = 1, 1
        
    elif shape == 'top-right':
        y_st, y_end, x_st, x_end = 0, i_max, j_max-1, -1
        y_sign, x_sign = 1, -1
        
    elif shape == 'bot-left':
        y_st, y_end, x_st, x_end = i_max-1, -1, 0, j_max
        y_sign, x_sign = -1 , 1
        
    else: # bot-right
        y_st, y_end, x_st, x_end = i_max-1, -1, j_max-1, -1
        y_sign, x_sign = -1 , -1
        #diag = True
     
    hit, total = 0, 0
    for i in range(y_st, y_end, y_sign):   
        if diag:
            print('i ', i)
        for j in range(x_st, x_end, x_sign):
            if diag:
                print('j ', j)
            if diag:
                print(surface[i, j])
            if surface[i, j] == val:
                hit += 1    
            total += 1
        x_end -= x_sign
        if (x_end - x_st) * x_sign <= -1:
            break
            
    return (hit, total)
        

In [326]:
def pairing(set1, set2, thresh, val):
    pair_dict = OrderedDict()
    perc_dict = OrderedDict()
    reject1_list, reject2_list = [], []
    for val1 in set1:
        x1 , y1 = val1
        match = False
        if len(set2) == 0:
            break
        for val2 in set2:
            hit, total = 0, 0
            x2 , y2 = val2
            if x1 >= x2 or y1 >= y2:
                continue
            perimeter = 2 * (y2 - y1+1) + 2 * (x2 - x1+1)
            top_horiz = thresh[y1, x1:x2+1]
            bot_horiz = thresh[y2, x1:x2+1]
            left_vert = thresh[y1:y2+1, x1]
            right_vert = thresh[y1:y2+1, x2]
            dot_array = np.hstack([top_horiz, bot_horiz, left_vert, right_vert])
            count_dots = np.count_nonzero(dot_array == val)
            cut_img = thresh[y1:y2+1 , x1:x2+1]
            # if the two coords are connected by rectangle then process...
            if count_dots >= perimeter * RECT_PERC:
                area = len(left_vert) * len(top_horiz)
                # if these points are in dark block then ignore them; or else, proceed...
                if is_block(cut_img, area, val):
                    pass
                else:
                    #if val1 == (561, 2662):                    
                    hit, total = count_dark_pixels(cut_img, val)
                    #print('cut ', cut_img.shape)
                    # proceed only if its a rectangular connection between the coords.. if it's curvy, then most likely
                    # it would be some character.. for curves, triangular arcs at 4 edges will be dark
                    if hit < total * ARC_PERC:
                        # if area is dense with dark pixels then it's not a rectangle
                        if dense_area(cut_img, val):
                            pass
                        else:
                            match = True
                            pair_dict[val1] = val2
                            perc_dict[val1] = (val2, float(count_dots) / perimeter)
                            print(val1, val2, perimeter, count_dots)                      
                    else:
                        print(val1, val2, perimeter, count_dots)
                        print('Dark ARC... ', hit, total)     
                break
                    #else:
                    #    print('no ARC... ', hit, total)     
        if val1 == (2450,2827):
            print('mark..', val2, count_dots, perimeter, hit, total, match)
        if match:
            set2.remove(val2)
            match = False
        else:
            reject1_list.append(val1)
    
    reject2_list.extend(set2)
    
    print(len(reject1_list), reject1_list,'\n')
    print(len(reject2_list), reject2_list,'\n')
    print(perc_dict, '\n')
    
    return pair_dict
        

In [337]:
#coord_l
final_dict = pairing(set1.copy(), set2.copy(), thresh, val)
#print(final_dict)


(113, 297) (1114, 436) 2284 2201
2 [(114, 381), (417, 126)] 

1 [(1115, 355)] 

OrderedDict([((113, 297), ((1114, 436), 0.96366024518388793))]) 



In [338]:
print(final_dict)

OrderedDict([((113, 297), (1114, 436))])


In [328]:
print(len(final), sorted(final),'\n')
print(len(final_l), sorted(final_l))

3 [(113, 297), (114, 381), (417, 126)] 

2 [(1114, 436), (1115, 355)]


In [329]:
start = final_dict.keys()
end = final_dict.values()
len(start), len(end) , start, end

(1, 1, odict_keys([(113, 297)]), odict_values([(1114, 436)]))

In [339]:
paint = [7,7]
for (x, y) in start:
    cv2.rectangle(img2,(x,y), (x+paint[0],  y+paint[1]), (110,110,110),3)
   
    #cv2.rectangle(img2,(x,y), (x+h_span, y+v_span), (110,110,110),3)
cv2.rectangle(img2,(1115, 355), (1115+paint[0],  355+paint[1]), (110,110,110),3)
   
cv2.imwrite('final_sample_loan01.jpg', img2)

True

In [331]:
paint = [7,7]
for (x, y) in end:
    cv2.rectangle(img2,(x,y), (x-paint[0],  y-paint[1]), (110,110,110),3)
    #cv2.rectangle(img2,(x,y), (x+h_span, y+v_span), (110,110,110),3)
cv2.imwrite('final_sample_loan01.jpg', img2)

True

# Cut, Fillter & Save

In [332]:
merg=zip(start,end)

In [333]:
crop=[]
for k,v in merg:
    x1,y1=k
    x2,y2=v
    #print(x1,y1,x2,y2)
    
    w=abs(x2-x1)
    h=abs(y2-y1)
    #print('x=',x1,'y=',y1,'w=',abs(x2-x1),'h=',abs(y2-y1),'\n')
    crop.append(img[y1+2:y1+h-2,x1+2:x1+w-2])  

In [334]:
(crop[0])

array([[  0, 255, 255, ..., 255, 255, 255],
       [  0,   0, 255, ..., 255,   0,   0],
       [  0,   0, 255, ..., 255,   0,   0],
       ..., 
       [  0,   0, 255, ..., 255, 255, 255],
       [  0,   0, 255, ..., 255, 255, 255],
       [  0,   0, 255, ..., 255, 255, 255]], dtype=uint8)

In [335]:
k=0
for i in crop:
    #len(crop)
    cv2.imwrite("img0"+str(k)+'.jpg',i)
    #cv2.imshow('imag',i)
    #cv2.waitKey(1000)
    #cv2.destroyAllWindows()
    k+=1

In [336]:
end_code

NameError: name 'end_code' is not defined

# fILLTER

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport new_filter


In [ ]:
k=0
for i in (crop):
    im=new_filter.main(i)
    cv2.imwrite('img0'+str(k)+'.jpg',im)
    k+=1

# finish

In [ ]:
finish

In [ ]:
#cut = thresh[2635:2654, 2002:2043].copy()
cut = thresh[2827:2850, 2450:2493].copy()
#cut = thresh[3259:3295, 701:736].copy()
#cut = thresh[4177:4233, 2005:2067].copy()
cv2.imshow('cut', cut)
cv2.waitKey(10000)
cv2.destroyAllWindows()


In [ ]:
x = cut.ravel()
np.count_nonzero(x == 255),  np.count_nonzero(x == 0), len(x)